# Darktide Media and Twitter Engagement
### Joseph Rush

On November 30th, 2022, Fatshark Games will be releasing Warhammer 40k: Darktide, an FPS PvE game and the third installment in its "Tide" series. On November 17th at 1300 EST, they will be releasing the closed pre-order beta. I set out to analyze the magnitude of interactions with their social media presence in the week leading up to the pre-order release. I also looked at the quantity of Darktide-related tweets coming from sources other than their primary @Darktide40k account as a measure of community "hype."

Which, if any, recent Darktide social media activity generated the most engagement in terms of likes and retweets? How did retweets of the main Darktide account compare to generation of original tweets by the wider community and media? How many of these accounts were interacting with each other around the subject of Darktide?

I first built a wide-ranging query that would return tweets with the key word "Darktide," that contained the marketing hashtags #darktide or #rejectswillrise, or that was from the main @Darktide40k account. This would gather both the official and community activity.

In [528]:
import pandas as pd
import json
import requests
import urllib

In [529]:
endpoint = 'https://api.twitter.com/2/tweets/search/recent'
bt = pd.read_csv('220922-TwitterBearerToken.txt', header = 0)

In [530]:
header = {'Authorization': 'Bearer {}'.format(bt['Bearer_Token'][0])}

In [531]:
query_text = 'darktide OR (#darktide OR #rejectswillrise) OR (from:darktide40k)'
query_encoded = urllib.parse.quote(query_text)
tweet_fields = 'author_id,created_at,public_metrics'
user_fields = 'name'
expansions = 'author_id'

In [532]:
query_url = endpoint + '?query={}&user.fields={}&tweet.fields={}&expansions={}'.format(query_encoded,user_fields,tweet_fields,expansions)
query_url

'https://api.twitter.com/2/tweets/search/recent?query=darktide%20OR%20%28%23darktide%20OR%20%23rejectswillrise%29%20OR%20%28from%3Adarktide40k%29&user.fields=name&tweet.fields=author_id,created_at,public_metrics&expansions=author_id'

In [541]:
def twt_darktide_search (query, num_pages,header):
    response_list = []
    next_token = ''
    for i in range(0, num_pages):
        if i>0:
            this_query = query + "&next_token={}".format(next_token)
        else:
            this_query = query
            
        this_response = requests.get(this_query, headers = header)
        #print(str(i) + " " + str(this_response.status_code))
        this_response_dict = json.loads(this_response.text)
        response_list.append(this_response_dict)
        next_token = this_response_dict['meta']['next_token']
        
    return response_list

In [542]:
my_responses = twt_darktide_search(query_url, 150, header)

In [535]:
results_1 = pd.DataFrame.from_records(my_responses)

In [536]:
results_1

,data,includes,meta
0,[{'text': 'RT @Darktide40K_JP: 11月11日は #独身の日 ！...,"{'users': [{'id': '1493470539423227907', 'name...","{'newest_id': '1593218995528032256', 'oldest_i..."
1,[{'edit_history_tweet_ids': ['1593217366229659...,"{'users': [{'id': '1578774917763301376', 'name...","{'newest_id': '1593217366229659648', 'oldest_i..."
2,"[{'author_id': '1555962195154010113', 'id': '1...","{'users': [{'id': '1555962195154010113', 'name...","{'newest_id': '1593215907983405056', 'oldest_i..."
3,"[{'author_id': '1569756545063469056', 'edit_hi...","{'users': [{'id': '1569756545063469056', 'name...","{'newest_id': '1593214087097454594', 'oldest_i..."
4,"[{'public_metrics': {'retweet_count': 0, 'repl...","{'users': [{'id': '3252467095', 'name': 'Hatas...","{'newest_id': '1593211915488354305', 'oldest_i..."
...,...,...,...
145,"[{'author_id': '78252816', 'edit_history_tweet...","{'users': [{'id': '78252816', 'name': 'Disco',...","{'newest_id': '1592805979082883073', 'oldest_i..."
146,[{'text': 'Darktide配信はありまぁす(*‘ω‘ *) ただ金曜やるとは限ら...,"{'users': [{'id': '1001209497640038400', 'name...","{'newest_id': '1592803224817631233', 'oldest_i..."
147,"[{'public_metrics': {'retweet_count': 0, 'repl...","{'users': [{'id': '1337486660527738880', 'name...","{'newest_id': '1592798985982730240', 'oldest_i..."
148,[{'edit_history_tweet_ids': ['1592794950525014...,"{'users': [{'id': '819618434124234753', 'name'...","{'newest_id': '1592794950525014018', 'oldest_i..."


## Comment

The data coming out of the preceding function is arranged in a nonsensical way for reasons I don't understand. The Data items aren't consistent. Sometimes Text is the first thing, sometimes Author_ID, other times Public_Metrics, etc. I don't understand what's happening here and it's preventing me from proceeding with my code.

Also, there's not enough of it. Each of those queries should be generating 10 tweets and then I am grabbing 150 pages of 10 tweets, but I only have 150 results. Something may be overwriting somewhere in a way I don't understand?

## End Comment

Once I had my tweet data, I needed to rearrange it.
The main items I needed for a tweet were:
1. Author ID
2. Author name
3. The text of the tweet
4. Retweet count
5. Reply count
6. Like count
7. Quote Count
8. Date created
9. The ID of the tweet itself

I then created code to pull these items out and arrange them neatly for export into a csv, putting each tweet on its own row and those nine items into columns

First, I needed my keys in the data column so I could pull out the desired items.

Once I had those I could start pulling out the items I needed.

In [543]:
#ARRANGING CODE into ArrangedResults dataframe
#Needs to arrange into the following columns:
#1. id from 'Data'
#2. text from 'Data'
#3. name from 'includes''users'
#5. reply_count from 'Data'
#6. retweet_count from 'Data'
#7. like_count from 'Data'
#8. quote_count from 'Data'
#9. created_at from 'Data'

#The Data Frame would be represented here using the .head() function to demonstrate its arrangement.

Once that was all arranged, the created ArrangedResults dataframe could be exported to csv so that I was not spamming the Twitter API or generating new data constantly.

In [ ]:
Arranged_Results_csv = ArrangedResults.to_csv()

The data collected is a good representation of Twitter interactions within its limits. It represents both official Fatshark activity as well as the media and community, and measures the impact and spread both of these sources are having across the platform. It generates accurate data about interactions and content on the Twitter platform itself.

You can analyze how many people are interacting with the community posts as well as the official posts. By including retweet data, the query allows an analysis of which accounts interact with the Fatshark account, which interact with community content, and any overlap or separation between the two.

In addition, the date & time information allows you to track this engagement over time and see if particular tweets, particularly those from the official account, may have been related to more community activity as well as analyzing which official tweets generated the most direct interaction. You can also construct a timeline of overall activity by volume, allowing a crude tracking of "hype" levels leading up to the release.

However, there are a few limitations worth noting.

First, the data is limited to 1500 tweets by my level of Twitter API access, since it cuts off at 1500 or a week of time, whichever comes first.

Second, the data itself is limited. It represents only Twitter, which is a small part of the social media footprint of a company and Twitter is not even a particularly strong driver of click-through and traffic. Fatshark's and Darktide's other social media and Discord activity is not represented here. The sample size of a Twitter audience is limited and Twitter is not a growing platform.

Third, the data does not include information about click-through and traffic generation to Fatshark's official blog and media posts that are advertised on their Twitter page. Twitter interaction is all well and good, but knowing how many users are drawn in by the Tweet and take the next step is vital to understanding the effect of a social media campaign.